Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [49]:
import numpy as np

In [50]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [51]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 3
NUM_DIMENSIONS = 2

In [52]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [53]:
WEIGHTS

array([[99, 92],
       [46, 97],
       [26, 52]], dtype=int32)

In [54]:
CONSTRAINTS

array([[ 86, 101],
       [ 39,  91]], dtype=int32)

In [55]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) <0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [56]:
solution

array([[ True, False,  True],
       [False,  True,  True]])

In [57]:
#variable to limit the number of attempts to find a valid configuration, used in both functions for the single knapsack and for all knapsacks
#avoid this value: definitely too long wait (with 1000 problem 3 took around 30 minutes to complete, so I'd use it just for problem 1)
approximation_limit = 10000

In [58]:
approximation_limit = 1000

In [59]:
approximation_limit = 100

In [60]:
def checkKnapsack(solution, knapsack):
    #checks if the solution is valid
    return np.all(WEIGHTS[solution[knapsack]].sum(axis=0) <= CONSTRAINTS[knapsack])

def fillKnapsack(knapsacks, numKnapsack, mask):
    #process: get all the values that can be chosen, get a random set of items to put in the knapsack
    #         and then check if the solution is valid

    for j in range(approximation_limit):
        
        #checking if there are still items to choose from
        if(np.sum(~mask) == 0):
            break
        #choosing knapsack items
        chosen_items = np.random.choice(np.where(~mask)[0], size=np.random.randint(1, np.sum(~mask)+1), replace=False)

        #creating the mask for the knapsack to be filled
        new_mask = np.zeros_like(mask, dtype=bool)
        new_mask[chosen_items] = np.True_

        #creating a copy of the knapsacks to test the new configuration
        knapsackCopy = knapsacks.copy()
        knapsackCopy[numKnapsack] |= new_mask

        #checking if the knapsack is valid
        if checkKnapsack(knapsackCopy, numKnapsack):
            return new_mask
        

    return np.zeros_like(mask, dtype=bool)
        
def calculateValue(solution):
    total_value = 0
    for k in range(NUM_KNAPSACKS):
        total_value += VALUES[solution[k]].sum()
    return total_value

def createSolution():

    #Variables used for tracking an approximate solution
    best_solution = None
    best_solution_value = 0

    for k in range(approximation_limit):  # Outer loop to restart the for loop

        #new solution initialization
        solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
        mask = np.zeros(NUM_ITEMS, dtype=bool)

        #loop to fill knapsacks
        for j in range(NUM_KNAPSACKS):

            # Filling the j-th knapsack
            solution[j] = fillKnapsack(solution, j, mask)

            # Condition to continue the loop, if the knapsack obtained is not empty probably there's still objects for other knapsacks
            if solution[j].sum() != 0:  
                mask |= solution[j]
                if(j == NUM_KNAPSACKS - 1):
                    
                    #checking if the solution is better than the best one found yet
                    value = calculateValue(solution)
                    if best_solution is None or value > best_solution_value:
                        best_solution = solution.copy()
                        best_solution_value = value
                continue
            
            value = calculateValue(solution)
            if best_solution is None or value > best_solution_value:
                best_solution = solution.copy()
                best_solution_value = value
            
            # Exit the for loop to restart
            break  
    # Return the best solution found
    print("Best solution with value ", best_solution_value)
    return best_solution  


In [61]:

solution = createSolution()

solution

Best solution with value  109


array([[False,  True, False],
       [False, False,  True]])

In [62]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

## TEST PROBLEMS

In [63]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [64]:

solution = createSolution()

solution

Best solution with value  1014


array([[False, False,  True,  True, False,  True, False, False,  True,
         True, False,  True, False, False,  True,  True, False,  True,
        False, False],
       [False,  True, False, False,  True, False,  True, False, False,
        False,  True, False,  True,  True, False, False, False, False,
        False, False],
       [ True, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
         True,  True]])

In [65]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [66]:

solution = createSolution()

solution

Best solution with value  35188


array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
         True, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
         True],
       [False, False, False, False, False,  True, False, False,  True,
        False, False, False, False, False,  True, False, False, False,
         True, False, False, False, False, False, False, Fals

In [67]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [68]:

solution = createSolution()

solution

Best solution with value  300638


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(100, 5000))